# Q2: Data Cleaning

**Phase 3:** Data Cleaning & Preprocessing  
**Points: 9 points**

**Focus:** Handle missing data, outliers, validate data types, remove duplicates.

**Lecture Reference:** Lecture 11, Notebook 1 ([`11/demo/01_setup_exploration_cleaning.ipynb`](https://github.com/christopherseaman/datasci_217/blob/main/11/demo/01_setup_exploration_cleaning.ipynb)), Phase 3. Also see Lecture 05 (data cleaning).

---

## Setup

In [6]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Load data from Q1 (or directly from source)
df_beach = pd.read_csv('data/beach_sensors.csv')
# If you saved cleaned data from Q1, you can load it:
# df = pd.read_csv('output/q1_exploration.csv')  # This won't work - load original

---

## Objective

Clean the dataset by handling missing data, outliers, validating data types, and removing duplicates.

**Time Series Note:** For time series data, forward-fill (`ffill()`) is often appropriate for missing values since sensor readings are continuous. However, you may choose other strategies based on your analysis.

---

## Required Artifacts

You must create exactly these 3 files in the `output/` directory:

### 1. `output/q2_cleaned_data.csv`
**Format:** CSV file
**Content:** Cleaned dataset with same structure as original (same columns)
**Requirements:**
- Same columns as original dataset
- Missing values handled (filled, dropped, or imputed)
- Outliers handled (removed, capped, or transformed)
- Data types validated and converted
- Duplicates removed
- **Sanity check:** Dataset should retain most rows after cleaning (at least 1,000 rows). If you're removing more than 50% of data, reconsider your strategy—imputation is usually preferable to dropping rows for this dataset.
- **No index column** (save with `index=False`)

### 2. `output/q2_cleaning_report.txt`
**Format:** Plain text file
**Content:** Detailed report of cleaning operations
**Required information:**
- Rows before cleaning: [number]
- Missing data handling method: [description]
  - Which columns had missing data
  - Method used (drop, forward-fill, impute, etc.)
  - Number of values handled
- Outlier handling: [description]
  - Detection method (IQR, z-scores, domain knowledge)
  - Which columns had outliers
  - Method used (remove, cap, transform)
  - Number of outliers handled
- Duplicates removed: [number]
- Data type conversions: [list any conversions]
- Rows after cleaning: [number]

**Example format:**
```
DATA CLEANING REPORT
====================

Rows before cleaning: 50000

Missing Data Handling:
- Water Temperature: 2500 missing values (5.0%)
  Method: Forward-fill (time series appropriate)
  Result: All missing values filled
  
- Air Temperature: 1500 missing values (3.0%)
  Method: Forward-fill, then median imputation for remaining
  Result: All missing values filled

Outlier Handling:
- Water Temperature: Detected 500 outliers using IQR method (3×IQR)
  Method: Capped at bounds [Q1 - 3×IQR, Q3 + 3×IQR]
  Bounds: [-5.2, 35.8]
  Result: 500 values capped

Duplicates Removed: 0

Data Type Conversions:
- Measurement Timestamp: Converted to datetime64[ns]

Rows after cleaning: 50000
```

### 3. `output/q2_rows_cleaned.txt`
**Format:** Plain text file
**Content:** Single integer number (total rows after cleaning)
**Requirements:**
- Only the number, no text, no labels
- No whitespace before or after
- Example: `50000`

---

## Requirements Checklist

- [ ] Missing data handling strategy chosen and implemented
- [ ] Outliers detected and handled (IQR method, z-scores, or domain knowledge)
- [ ] Data types validated and converted
- [ ] Duplicates identified and removed
- [ ] Cleaning decisions documented in report
- [ ] All 3 required artifacts saved with exact filenames

---

## Your Approach

1. **Handle missing data** - Choose appropriate strategy (drop, forward-fill, impute) based on data characteristics
2. **Detect and handle outliers** - Use IQR method or z-scores; decide whether to remove, cap, or transform
3. **Validate data types** - Ensure numeric and datetime columns are properly typed
4. **Remove duplicates**
5. **Document and save** - Write detailed cleaning report explaining your decisions

---

## Decision Points

- **Missing data:** Should you drop rows, impute values, or forward-fill? Consider: How much data is missing? Is it random or systematic? For time series, forward-fill is often appropriate.
- **Outliers:** Are they errors or valid extreme values? Use IQR method or z-scores to detect, then decide: remove, cap, or transform. Document your reasoning.
- **Data types:** Are numeric columns actually numeric? Are datetime columns properly formatted? Convert as needed.

---

## Checkpoint

After Q2, you should have:
- [ ] Missing data handled
- [ ] Outliers addressed
- [ ] Data types validated
- [ ] Duplicates removed
- [ ] All 3 artifacts saved: `q2_cleaned_data.csv`, `q2_cleaning_report.txt`, `q2_rows_cleaned.txt`

---

**Next:** Continue to `q3_data_wrangling.md` for Data Wrangling.


In [7]:
import os

#Missing Data by column
display("# Missing Values")

missing = df_beach.isnull().sum()
missing_pct = (missing / len(df_beach)) *100
missing_df = pd.DataFrame({
    'Missing Count' : missing, 
    'Missing %': missing_pct.round(2)
})
missing_df = missing_df[missing_df['Missing Count'] > 0].sort_values('Missing Count', ascending=False)

if len(missing_df) == 0:
    display("**No missing values found!**")
else:
    display(missing_df)

cols_missing = ["Wet Bulb Temperature","Rain Intensity","Total Rain","Heading","Precipitation Type","Barometric Pressure","Air Temperature"]

df_beach[cols_missing] = df_beach[cols_missing].ffill(limit=3).bfill(limit=3)
df_beach[cols_missing] = df_beach[cols_missing].fillna(df_beach[cols_missing].median())
print(df_beach.isna().sum())


display("# Outlier Detection")

# Identify outliers using percentile technique (0.1–99.9th percentiles)

numeric_cols = df_beach.select_dtypes(include='number').columns

outliers_initial = {}      # per-column info (counts + bounds)
outlier_indices = set()    # unique row indices with at least one outlier

for col in numeric_cols:
    lower_bound = df_beach[col].quantile(0.001)
    upper_bound = df_beach[col].quantile(0.999)
    
    # mask of ORIGINAL outliers
    mask = (df_beach[col] < lower_bound) | (df_beach[col] > upper_bound)
    col_indices = df_beach.index[mask]
    col_outlier_count = int(mask.sum())
    
    outlier_indices.update(col_indices)
    
    # store info for this column
    outliers_initial[col] = {
        "lower_bound": lower_bound,
        "upper_bound": upper_bound,
        "outlier_count": col_outlier_count,
        "indices": col_indices,
    }

# Cap (winsorize) outliers using the stored bounds
for col in numeric_cols:
    lower = outliers_initial[col]["lower_bound"]
    upper = outliers_initial[col]["upper_bound"]
    df_beach[col] = df_beach[col].clip(lower, upper)

# Build a summary DataFrame
outlier_summary = (
    pd.DataFrame.from_dict(outliers_initial, orient="index")
      .loc[:, ["outlier_count", "lower_bound", "upper_bound"]]
      .query("outlier_count > 0")
)
outlier_summary.index.name = "column_name"

display(outlier_summary)

# Pretty printed per-column summary, e.g.:
# Wind Speed: 2,574 outliers capped (bounds: [-3.50, 8.40])
for col, info in outliers_initial.items():
    if info["outlier_count"] > 0:
        print(
            f"{col}: {info['outlier_count']} outliers capped "
            f"(bounds: [{info['lower_bound']:.2f}, {info['upper_bound']:.2f}])"
        )

# Cleaned dataframe = capped version (no rows dropped)
df_beach_clean = df_beach.copy()

# Unique rows that had at least one outlier BEFORE capping
rows_with_outliers = len(outlier_indices)

print("Original rows:", df_beach.shape[0])
print("Rows with at least one outlier (before capping):", rows_with_outliers)
print("Cleaned rows (after capping):", df_beach_clean.shape[0])

df_beach_clean['Measurement Timestamp'] = pd.to_datetime(df_beach_clean['Measurement Timestamp'])
col_summary_clean = df_beach_clean.dtypes.to_frame(name="dtype")
print(col_summary_clean)


# Add description column for the text report
outlier_summary["description"] = (
    "- " + outlier_summary.index
    + ": " + outlier_summary["outlier_count"].astype(str)
    + " outliers detected and capped "
)

outlier_report = (
    "Outlier Handling:\n"
    "Detection method: Percentile-based thresholds (0.1–99.9th percentiles)\n"
    f"Columns with outliers: {', '.join(outlier_summary.index)}\n"
    "Method used: Capped outliers to percentile bounds (winsorization)\n"
    f"Number of rows with at least one outlier (before capping): {rows_with_outliers}\n\n"
    + "\n".join(outlier_summary["description"])
)


print("Original rows:", df_beach.shape[0])
print("Rows removed:", len(outlier_indices))
print("Cleaned rows:", df_beach_clean.shape[0])

df_beach_clean['Measurement Timestamp'] = pd.to_datetime(df_beach_clean['Measurement Timestamp'])
col_summary_clean = df_beach_clean.dtypes.to_frame(name = "dtype")
print(col_summary_clean)

# Check for duplicate rows
display("# Duplicate Detection")

# Check for completely duplicate rows
n_duplicates = df_beach_clean.duplicated().sum()
display(f"**Completely duplicate rows:** {n_duplicates:,}")


summary_clean = pd.DataFrame({
    "column_name": df_beach_clean.columns,
    "mean": df_beach_clean.mean(numeric_only=True).reindex(df_beach_clean.columns),
    "std": df_beach_clean.std(numeric_only=True).reindex(df_beach_clean.columns),
    "min": df_beach_clean.min(numeric_only=True).reindex(df_beach_clean.columns),
    "max": df_beach_clean.max(numeric_only=True).reindex(df_beach_clean.columns),
    "missing_count": df_beach_clean.isna().sum()
})

df_beach_clean.to_csv("output/q2_cleaned_data.csv", index=False)

missing = df_beach.isnull().sum()
missing_pct = (missing / len(df_beach)) *100

missing_before = pd.DataFrame({
    'Missing Count' : missing, 
    'Missing %': missing_pct.round(3)
})

def make_description(row):
    return (
        f"- {row.name}: {int(row['Missing Count'])} missing values ({row['Missing %']:.2f}%)\n"
        f"  Method: Forward-fill (time series appropriate)\n"
        f"  Result: All missing values filled"
    )

missing_df["description"] = missing_df.apply(make_description, axis=1)


with open("output/q2_cleaning_report.txt", "w") as f:
    f.write("DATA CLEANING REPORT \n ========== \n")
    f.write(f"Rows before cleaning: {df_beach.shape[0]} \n")
    f.write(f"Missing Data Handling: \n")
    for text in missing_df["description"]:
        f.write(text + "\n\n")
    f.write(outlier_report)
    f.write(f"\n Duplicates removed: {n_duplicates:,} \n")
    f.write(f"Measurement Timestamp: Converted to datetime64[ns]\n")
    f.write(f"Rows after cleaning: {df_beach_clean.shape[0]} \n")

with open("output/q2_rows_cleaned.txt", "w") as f:
    f.write(f"{df_beach_clean.shape[0]}")

'# Missing Values'

,Missing Count,Missing %
Wet Bulb Temperature,75951,38.69
Rain Intensity,75951,38.69
Total Rain,75951,38.69
Heading,75951,38.69
Precipitation Type,75951,38.69
Barometric Pressure,146,0.07
Air Temperature,75,0.04


Station Name                   0
Measurement Timestamp          0
Air Temperature                0
Wet Bulb Temperature           0
Humidity                       0
Rain Intensity                 0
Interval Rain                  0
Total Rain                     0
Precipitation Type             0
Wind Direction                 0
Wind Speed                     0
Maximum Wind Speed             0
Barometric Pressure            0
Solar Radiation                0
Heading                        0
Battery Life                   0
Measurement Timestamp Label    0
Measurement ID                 0
dtype: int64


'# Outlier Detection'

,outlier_count,lower_bound,upper_bound
column_name,,,
Air Temperature,377,-19.670,34.1000
Wet Bulb Temperature,390,-19.468,26.9000
Humidity,185,23.000,100.0000
Rain Intensity,188,0.000,22.2000
Interval Rain,198,0.000,15.0876
Total Rain,126,0.000,1021.0000
Wind Speed,191,0.000,12.4000
Maximum Wind Speed,193,0.000,17.4000
Barometric Pressure,367,967.500,1016.6000


Air Temperature: 377 outliers capped (bounds: [-19.67, 34.10])
Wet Bulb Temperature: 390 outliers capped (bounds: [-19.47, 26.90])
Humidity: 185 outliers capped (bounds: [23.00, 100.00])
Rain Intensity: 188 outliers capped (bounds: [0.00, 22.20])
Interval Rain: 198 outliers capped (bounds: [0.00, 15.09])
Total Rain: 126 outliers capped (bounds: [0.00, 1021.00])
Wind Speed: 191 outliers capped (bounds: [0.00, 12.40])
Maximum Wind Speed: 193 outliers capped (bounds: [0.00, 17.40])
Barometric Pressure: 367 outliers capped (bounds: [967.50, 1016.60])
Solar Radiation: 223 outliers capped (bounds: [-6.00, 964.00])
Battery Life: 275 outliers capped (bounds: [11.70, 15.20])
Original rows: 196321
Rows with at least one outlier (before capping): 2340
Cleaned rows (after capping): 196321
                                      dtype
Station Name                         object
Measurement Timestamp        datetime64[ns]
Air Temperature                     float64
Wet Bulb Temperature                

'# Duplicate Detection'

'**Completely duplicate rows:** 0'